### 3. 그룹바이 심화

### 3-1. 일반 접근방식

In [1]:
import pandas as pd

selloutData = pd.read_csv("../dataset/kopo_product_volume.csv")
selloutData.head()

selloutData.columns = selloutData.columns.str.lower()

sortedData = selloutData.sort_values(["regionid","productgroup","yearweek"])
sortedData.reset_index(inplace=True, drop=True)
sortedData.head()

sortedData["ma"] = sortedData["volume"].rolling(window=17, center=False, min_periods=1).mean()
sortedData.head()

,regionid,productgroup,yearweek,volume,ma
0,A01,ST0001,201401,513598,513598.000000
1,A01,ST0001,201402,438251,475924.500000
2,A01,ST0001,201403,420290,457379.666667
3,A01,ST0001,201404,458431,457642.500000
4,A01,ST0001,201405,482381,462590.200000


In [2]:
sortedData.to_csv("./group_test.csv")

### 3-2. 고급접근 방식

In [3]:
import pandas as pd

selloutData = pd.read_csv("../dataset/kopo_product_volume.csv")
selloutData.head()

selloutData.columns = selloutData.columns.str.lower()

sortedData = selloutData.sort_values(["regionid","productgroup","yearweek"])
sortedData.reset_index(inplace=True, drop=True)
sortedData.head()

groupResult = sortedData.groupby(["regionid","productgroup"])["volume"].mean()
groupResult = groupResult.reset_index()
groupResult.rename(columns={'volume': 'avg_volume'}, inplace=True)
groupResult.head(2)

,regionid,productgroup,avg_volume
0,A01,ST0001,672850.421053
1,A01,ST0002,189489.924242


In [4]:
finalResult = sortedData.merge(groupResult, how="left", 
                 on = ["regionid","productgroup"])
finalResult.head()

,regionid,productgroup,yearweek,volume,avg_volume
0,A01,ST0001,201401,513598,672850.421053
1,A01,ST0001,201402,438251,672850.421053
2,A01,ST0001,201403,420290,672850.421053
3,A01,ST0001,201404,458431,672850.421053
4,A01,ST0001,201405,482381,672850.421053


In [5]:
finalResult.to_csv("./groupResult.csv")

### 3-3 고급 그룹바이 함수 정의

In [30]:
def sub_function(data):
    data["ma"] = data["qty"].rolling(window = 5, center = True, min_periods=1).mean()
    return data

In [31]:
import pandas as pd

selloutData = pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")
selloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0
1,A60,PRODUCT59,201402,22275.0
2,A60,PRODUCT34,201402,4463.0
3,A60,PRODUCT47,201402,0.0
4,A60,PRODUCT56,201402,23.0


In [32]:
selloutData.columns = selloutData.columns.str.lower()
selloutData.head()

,regionid,product,yearweek,qty
0,A60,PRODUCT4,201402,71.0
1,A60,PRODUCT59,201402,22275.0
2,A60,PRODUCT34,201402,4463.0
3,A60,PRODUCT47,201402,0.0
4,A60,PRODUCT56,201402,23.0


In [33]:
sortedData = selloutData.sort_values(["regionid","product","yearweek"])
sortedData.head()

,regionid,product,yearweek,qty
298,A00,PRODUCT34,201401,661.0
1757,A00,PRODUCT34,201402,679.0
3125,A00,PRODUCT34,201403,578.0
205,A00,PRODUCT34,201404,532.0
4369,A00,PRODUCT34,201405,516.0


In [34]:
sortedData.reset_index(inplace=True, drop=True)
sortedData.head()

,regionid,product,yearweek,qty
0,A00,PRODUCT34,201401,661.0
1,A00,PRODUCT34,201402,679.0
2,A00,PRODUCT34,201403,578.0
3,A00,PRODUCT34,201404,532.0
4,A00,PRODUCT34,201405,516.0


In [35]:
groupData = sortedData.groupby(["regionid","product"])

In [36]:
len(groupData)

794

In [37]:
eachData = groupData.get_group(list(groupData.groups)[793])
eachData.head()

,regionid,product,yearweek,qty
124501,A77,PRODUCT12,201401,3821.0
124502,A77,PRODUCT12,201402,2466.0
124503,A77,PRODUCT12,201403,2401.0
124504,A77,PRODUCT12,201404,2628.0
124505,A77,PRODUCT12,201405,1442.0


### 함수 생성하기

In [38]:
period = 17

In [39]:
def ma_rolling(eachData, term):
    basicData = eachData.reset_index(drop=True)
    #basicData.head(2)

    basicData["ma"] = basicData["qty"].rolling(window=term, center=True, min_periods=1).mean()
    #basicData.head(2)
    return basicData

In [40]:
GroupResult = sortedData.groupby(["regionid","product"]).apply(ma_rolling, period)

In [41]:
GroupResult.head(2)

regionid    product  yearweek    qty          ma
regionid product                                                     
A00      PRODUCT34 0      A00  PRODUCT34    201401  661.0  514.444444
                   1      A00  PRODUCT34    201402  679.0  516.800000

In [42]:
aa =GroupResult.reset_index(drop=True)

In [43]:
aa["ratio"] = aa["qty"] / aa["ma"]
aa.head()

,regionid,product,yearweek,qty,ma,ratio
0,A00,PRODUCT34,201401,661.0,514.444444,1.284881
1,A00,PRODUCT34,201402,679.0,516.800000,1.313854
2,A00,PRODUCT34,201403,578.0,503.363636,1.148275
3,A00,PRODUCT34,201404,532.0,490.750000,1.084055
4,A00,PRODUCT34,201405,516.0,480.230769,1.074483


In [44]:
aa["week"] = aa["yearweek"].astype(str).str[4:]

In [45]:
aa.head(2)

,regionid,product,yearweek,qty,ma,ratio,week
0,A00,PRODUCT34,201401,661.0,514.444444,1.284881,01
1,A00,PRODUCT34,201402,679.0,516.800000,1.313854,02


In [46]:
len(eachData)

157

In [47]:
finalGroupResult = sortedData.groupby(["regionid","product"]).apply(sub_function)

In [49]:
selloutData.columns = selloutData.columns.str.lower()

sortedData = selloutData.sort_values(["regionid","product","yearweek"])
sortedData.reset_index(inplace=True, drop=True)
sortedData.head()

finalGroupResult = sortedData.groupby(["regionid","product"]).apply(sub_function)
finalGroupResult.head(3)

,regionid,product,yearweek,qty,ma
0,A00,PRODUCT34,201401,661.0,639.333333
1,A00,PRODUCT34,201402,679.0,612.500000
2,A00,PRODUCT34,201403,578.0,593.200000


In [51]:
finalGroupResult["year"] = finalGroupResult["yearweek"].astype(str).str[:4]
finalGroupResult.head(2)

,regionid,product,yearweek,qty,ma,year
0,A00,PRODUCT34,201401,661.0,639.333333,2014
1,A00,PRODUCT34,201402,679.0,612.500000,2014


In [52]:
finalGroupResult["week"] = finalGroupResult["yearweek"].astype(str).str[4:]
finalGroupResult.head(2)

,regionid,product,yearweek,qty,ma,year,week
0,A00,PRODUCT34,201401,661.0,639.333333,2014,01
1,A00,PRODUCT34,201402,679.0,612.500000,2014,02


In [53]:
finalGroupResult.to_csv("./finalGroupBy.csv")

### 3-3. 그룹바이 함수 디버깅

In [55]:
groupData = sortedData.groupby(["regionid","product"])

In [57]:
len(groupData)

794

In [58]:
data = groupData.get_group(list(groupData.groups)[0])
data.head()

,regionid,product,yearweek,qty
0,A00,PRODUCT34,201401,661.0
1,A00,PRODUCT34,201402,679.0
2,A00,PRODUCT34,201403,578.0
3,A00,PRODUCT34,201404,532.0
4,A00,PRODUCT34,201405,516.0


In [60]:
data["ma"] = data["qty"].rolling(window = 5, center = True, min_periods=1).mean()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [61]:
data.head()

,regionid,product,yearweek,qty,ma
0,A00,PRODUCT34,201401,661.0,639.333333
1,A00,PRODUCT34,201402,679.0,612.500000
2,A00,PRODUCT34,201403,578.0,593.200000
3,A00,PRODUCT34,201404,532.0,545.600000
4,A00,PRODUCT34,201405,516.0,491.200000
